In [1]:
import logging
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow as tf

import tensorflow_text
import boto3

2023-06-13 14:36:26.103374: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 14:36:26.766809: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [28]:
# load the dataset
examples, metadata = tfds.load('ted_hrlr_translate/tr_to_en',
                               with_info=True,
                               as_supervised=True)

train_examples, val_examples, test_examples = examples['train'], examples['validation'], examples['test']

#save dataset to csv
def save_to_csv(dataset, filename):
      with open(filename, 'a+') as f:
         for pt, en in tfds.as_numpy(dataset):
               f.write(str(pt, 'utf-8') + "<DELIM>" + str(en, 'utf-8') + "\n")

#save training, validation and test to one csv file
save_to_csv(train_examples, 'data.csv')
save_to_csv(val_examples, 'data.csv')
save_to_csv(test_examples, 'data.csv')

2023-06-13 15:03:03.179440: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]
2023-06-13 15:03:03.179969: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2023-06-13 15:03:35.940519: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{no

In [31]:
import pandas as pd
data_df = pd.read_csv('data.csv', names=['tr', 'en'], sep='<DELIM>')
train_df = pd.read_csv('train.csv', names=['tr', 'en'], sep='<DELIM>')
val_df = pd.read_csv('val.csv', names=['tr', 'en'], sep='<DELIM>')
test_df = pd.read_csv('test.csv', names=['tr', 'en'], sep='<DELIM>')

#print number of samples in each dataset
print('Number of training samples: ', len(data_df[data_df['tr'].notna()]))
print('Number of samples', len(train_df[train_df['tr'].notna()]))
print('Number of samples', len(val_df[val_df['tr'].notna()]))
print('Number of samples', len(test_df[test_df['tr'].notna()]))


/tmp/ipykernel_15079/3382394657.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data_df = pd.read_csv('data.csv', names=['tr', 'en'], sep='<DELIM>')
/tmp/ipykernel_15079/3382394657.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train_df = pd.read_csv('train.csv', names=['tr', 'en'], sep='<DELIM>')


Number of training samples:  383048
Number of samples 182450


In [26]:
print(train_df.head())
# print(train_df["tr"][0])
# print(train_df["en"][0])
print(test_df["en"][0])
print(test_df["tr"][0])

                                                  tr  \
0  emekli üyeler kongre'nin şu sıralar çete savaş...   
1  entellektüellik , klas , asalet veya hikaye il...   
2  hangisi olduğunu tahmin edebildiniz mi ? şirke...   
3  pek uzak yerlere seyahat edemez veya belli bir...   
4                                 heyecanlanmıştım .   

                                                  en  
0  retiring members nowadays say that it 's becom...  
1  no sophistication , no class , no dignity , no...  
2                     did you guess it ? companies .  
3  you ca n't travel very far or venture too far ...  
4                                    i was excited .  
myriad did n't even argue this , so it came out of the blue .
myriad bunu hiç savunmamıştı bile .


In [8]:
# testing if the dataset is loaded correctly
for tr_examples, en_examples in train_examples.batch(3).take(1):
  print('> Examples in Turkish:')
  for tr in tr_examples.numpy():
    print(tr.decode('utf-8'))
  print()

  print('> Examples in English:')
  for en in en_examples.numpy():
    print(en.decode('utf-8'))

> Examples in Turkish:
emekli üyeler kongre'nin şu sıralar çete savaşlarına döndüğünü söylüyorlar .
entellektüellik , klas , asalet veya hikaye ile ilgisi yok ,
hangisi olduğunu tahmin edebildiniz mi ? şirketler .

> Examples in English:
retiring members nowadays say that it 's become like gang warfare .
no sophistication , no class , no dignity , no history .
did you guess it ? companies .


2023-06-12 15:50:33.990539: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [1]
	 [[{{node Placeholder/_1}}]]
2023-06-12 15:50:33.991384: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2023-06-12 15:50:34.039198: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline 

In [ ]:
# Tokenize the dataset
bert_tokenizer_params=dict(lower_case=True)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_args = dict(
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)
